# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2023 11:18AM,264746,21,8009835,"NBTY Global, Inc.",Released
1,Jun 14 2023 11:02AM,264745,21,9572365-1,"NBTY Global, Inc.",Released
2,Jun 14 2023 10:47AM,264744,19,60021081,"GUSA Granules USA, Inc.",Released
3,Jun 14 2023 10:36AM,264742,19,60021072,"GUSA Granules USA, Inc.",Released
4,Jun 14 2023 10:36AM,264741,10,MSP222685,"Methapharm, Inc.",Released
5,Jun 14 2023 10:36AM,264741,10,MSP222686,"Methapharm, Inc.",Released
6,Jun 14 2023 10:36AM,264741,10,MSP222688,"Methapharm, Inc.",Released
7,Jun 14 2023 10:36AM,264741,10,MSP222693,"Methapharm, Inc.",Released
8,Jun 14 2023 10:36AM,264741,10,MSP222694,"Methapharm, Inc.",Released
9,Jun 14 2023 10:19AM,264740,19,N18123,"NAPP Technologies, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,264741,Released,5
32,264742,Released,1
33,264744,Released,1
34,264745,Released,1
35,264746,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264741,NaN,NaN,5.0
264742,NaN,NaN,1.0
264744,NaN,NaN,1.0
264745,NaN,NaN,1.0
264746,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,16.0,5.0,5.0
264685,0.0,1.0,2.0
264686,0.0,0.0,1.0
264687,0.0,0.0,2.0
264694,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,16,5,5
264685,0,1,2
264686,0,0,1
264687,0,0,2
264694,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,16,5,5
1,264685,0,1,2
2,264686,0,0,1
3,264687,0,0,2
4,264694,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,16,5,5
1,264685,,1,2
2,264686,,,1
3,264687,,,2
4,264694,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc."
1,Jun 14 2023 11:02AM,264745,21,"NBTY Global, Inc."
2,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc."
3,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc."
4,Jun 14 2023 10:36AM,264741,10,"Methapharm, Inc."
9,Jun 14 2023 10:19AM,264740,19,"NAPP Technologies, LLC"
10,Jun 14 2023 10:09AM,264738,21,"NBTY Global, Inc."
11,Jun 14 2023 10:06AM,264736,15,Lonza
12,Jun 14 2023 10:02AM,264734,15,"Emersa Waterbox, LLC"
13,Jun 14 2023 10:00AM,264733,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",,,1
1,Jun 14 2023 11:02AM,264745,21,"NBTY Global, Inc.",,,1
2,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",,,1
3,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",,,1
4,Jun 14 2023 10:36AM,264741,10,"Methapharm, Inc.",,,5
5,Jun 14 2023 10:19AM,264740,19,"NAPP Technologies, LLC",,,1
6,Jun 14 2023 10:09AM,264738,21,"NBTY Global, Inc.",,,1
7,Jun 14 2023 10:06AM,264736,15,Lonza,,1,
8,Jun 14 2023 10:02AM,264734,15,"Emersa Waterbox, LLC",,,1
9,Jun 14 2023 10:00AM,264733,10,Eywa Pharma Inc.,,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",1,,
1,Jun 14 2023 11:02AM,264745,21,"NBTY Global, Inc.",1,,
2,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",1,,
3,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",1,,
4,Jun 14 2023 10:36AM,264741,10,"Methapharm, Inc.",5,,
5,Jun 14 2023 10:19AM,264740,19,"NAPP Technologies, LLC",1,,
6,Jun 14 2023 10:09AM,264738,21,"NBTY Global, Inc.",1,,
7,Jun 14 2023 10:06AM,264736,15,Lonza,,1,
8,Jun 14 2023 10:02AM,264734,15,"Emersa Waterbox, LLC",1,,
9,Jun 14 2023 10:00AM,264733,10,Eywa Pharma Inc.,13,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",1,,
1,Jun 14 2023 11:02AM,264745,21,"NBTY Global, Inc.",1,,
2,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",1,,
3,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",1,,
4,Jun 14 2023 10:36AM,264741,10,"Methapharm, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 14 2023 11:02AM,264745,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Jun 14 2023 10:36AM,264741,10,"Methapharm, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 14 2023 11:02AM,264745,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Jun 14 2023 10:36AM,264741,10,"Methapharm, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3441316,52.0,14.0,0.0
15,529470,1.0,1.0,0.0
19,2117685,18.0,17.0,16.0
20,1058846,8.0,1.0,0.0
21,1058961,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3441316,52.0,14.0,0.0
1,15,529470,1.0,1.0,0.0
2,19,2117685,18.0,17.0,16.0
3,20,1058846,8.0,1.0,0.0
4,21,1058961,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,14.0,0.0
1,15,1.0,1.0,0.0
2,19,18.0,17.0,16.0
3,20,8.0,1.0,0.0
4,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,15,Released,1.0
2,19,Released,18.0
3,20,Released,8.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,0.0,16.0,0.0,0.0
Executing,14.0,1.0,17.0,1.0,1.0
Released,52.0,1.0,18.0,8.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,0.0,16.0,0.0,0.0
1,Executing,14.0,1.0,17.0,1.0,1.0
2,Released,52.0,1.0,18.0,8.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,0.0,16.0,0.0,0.0
1,Executing,14.0,1.0,17.0,1.0,1.0
2,Released,52.0,1.0,18.0,8.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()